In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pandas_profiling import ProfileReport

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import sklearn.metrics as met
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import RFE

from scipy import stats

## Importing Data

In [5]:
# behaviour = pd.read_csv('CustomerBehaviorData.csv')
# consumption = pd.read_csv('CreditConsumptionData.csv')
# customer = pd.read_csv('CustomerDemographics.csv')

In [6]:
behaviour = pd.read_csv('https://github.com/lalit-kumr/credit_card_consumption/blob/main/CustomerBehaviorData.csv?raw=true')
consumption = pd.read_csv('https://github.com/lalit-kumr/credit_card_consumption/raw/main/CreditConsumptionData.csv')
customer = pd.read_csv('https://github.com/lalit-kumr/credit_card_consumption/raw/main/CustomerDemographics.csv')

### Cleaning Customer df

In [7]:
#finding NaN values in customer df
customer.isna().sum()

customer_id                     0
account_type                    1
gender                          1
age                             0
income                          1
emp_tenure_years                0
tenure_with_bank                0
region_code                     1
net_banking_flag                0
avg_days_between_transaction    3
dtype: int64

In [8]:
#Droppping observations having NaN values
customer.dropna(inplace=True)
customer.isna().sum()

customer_id                     0
account_type                    0
gender                          0
age                             0
income                          0
emp_tenure_years                0
tenure_with_bank                0
region_code                     0
net_banking_flag                0
avg_days_between_transaction    0
dtype: int64

### Cleaning behaviour df

In [9]:
behaviour.isna().sum()

customer_id              0
cc_cons_apr              0
dc_cons_apr              0
cc_cons_may              1
dc_cons_may              0
cc_cons_jun              0
dc_cons_jun              1
cc_count_apr             1
cc_count_may             0
cc_count_jun             0
dc_count_apr             0
dc_count_may             0
dc_count_jun             0
card_lim                 0
personal_loan_active     0
vehicle_loan_active      0
personal_loan_closed     1
vehicle_loan_closed      0
investment_1             0
investment_2             0
investment_3             2
investment_4             0
debit_amount_apr         0
credit_amount_apr        0
debit_count_apr          1
credit_count_apr         0
max_credit_amount_apr    0
debit_amount_may         0
credit_amount_may        0
credit_count_may         0
debit_count_may          0
max_credit_amount_may    0
debit_amount_jun         0
credit_amount_jun        0
credit_count_jun         0
debit_count_jun          0
max_credit_amount_jun    0
l

In [10]:
#Droppping observations having NaN values
behaviour.dropna(inplace=True)
behaviour.isna().sum()

customer_id              0
cc_cons_apr              0
dc_cons_apr              0
cc_cons_may              0
dc_cons_may              0
cc_cons_jun              0
dc_cons_jun              0
cc_count_apr             0
cc_count_may             0
cc_count_jun             0
dc_count_apr             0
dc_count_may             0
dc_count_jun             0
card_lim                 0
personal_loan_active     0
vehicle_loan_active      0
personal_loan_closed     0
vehicle_loan_closed      0
investment_1             0
investment_2             0
investment_3             0
investment_4             0
debit_amount_apr         0
credit_amount_apr        0
debit_count_apr          0
credit_count_apr         0
max_credit_amount_apr    0
debit_amount_may         0
credit_amount_may        0
credit_count_may         0
debit_count_may          0
max_credit_amount_may    0
debit_amount_jun         0
credit_amount_jun        0
credit_count_jun         0
debit_count_jun          0
max_credit_amount_jun    0
l

### Cleaning consumption df

In [11]:
consumption.isna().sum() # no NaN values except the ones we have to predict

customer_id          0
cc_cons_target    5000
dtype: int64

### Merging the dataframes 
- now we will merge all three dataframes on the basis of customer_id.
- first we will merge customer and behaviour dataframes as customer_be
- next we will merge the cosumption dataframe with the custoemr_be df and store the resultant dataframe in **final**

In [127]:
customer_behaviour = pd.merge(left=customer, right=behaviour, on='customer_id')
final = pd.merge(left=customer_behaviour, right=consumption, on='customer_id')
final.head(2)

,customer_id,account_type,gender,age,income,emp_tenure_years,tenure_with_bank,region_code,net_banking_flag,avg_days_between_transaction,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,dc_count_apr,dc_count_may,dc_count_jun,card_lim,personal_loan_active,vehicle_loan_active,personal_loan_closed,vehicle_loan_closed,investment_1,investment_2,investment_3,investment_4,debit_amount_apr,credit_amount_apr,debit_count_apr,credit_count_apr,max_credit_amount_apr,debit_amount_may,credit_amount_may,credit_count_may,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,loan_enq,emi_active,cc_cons_target
0,19427,current,M,63,MEDIUM,30.1,10,628.0,1,5.0,7998.48,2289.0,9553.00,2879.58,6881.81,3124.0,8.0,23,13,47,3,30,519000,1,1,1.0,1,13659.0,14000.0,80000.0,6885.0,79085.00,44630.50,63.0,1,48134.0,107662.02,61887.50,5,14,20770.0,44884.90,369000.75,40,96,46088.0,Y,2646.72,NaN
1,16150,current,M,36,MEDIUM,14.4,10,656.0,0,12.0,16479.64,415.0,7386.49,6763.00,12032.60,6738.0,19.0,32,7,1,37,16,402000,1,1,1.0,1,942327.7,172131.0,28912.0,553.0,85171.16,216770.95,10.0,1,31820.0,34376.79,36644.25,4,63,78627.0,91073.84,243182.32,7,12,17953.0,Y,5469.79,NaN


In [128]:
#Total investments
final['investment'] = final['investment_1']+final['investment_2']+final['investment_3']+final['investment_4']
#Total debit amount
final["Total_debit_amount"]=final['debit_amount_apr']+final['debit_amount_may']+final['debit_amount_jun']
#Total Credit Amount
final["Total_credit_amount"]=final['credit_amount_apr']+final['credit_amount_may']+final['credit_amount_jun']
#Total Max Credit amount
final["Total_max_credit_acmout"]=final['max_credit_amount_apr']+final['max_credit_amount_may']+final['max_credit_amount_jun']
#Total Active Loan
final["Totat_active_loan"]= final['personal_loan_active'] + final['vehicle_loan_active']
#Toatal Closed Loan
final["Total_closed_loan"]= final['personal_loan_closed'] + final['vehicle_loan_closed']
#Credit Card expenditure total
final['creditcard_exp']= final['cc_cons_apr'] + final['cc_cons_may'] + final['cc_cons_jun'] 
#Debit Card expenditure total
final['debitcard_exp']= final['dc_cons_apr'] + final['dc_cons_may'] + final['dc_cons_jun']




In [129]:
extra = ['investment_1','investment_2','investment_3','investment_4',
         'debit_amount_apr','debit_amount_may','debit_amount_jun',
         'credit_amount_apr','credit_amount_may','credit_amount_jun',
         'max_credit_amount_apr','max_credit_amount_may','max_credit_amount_jun',
         'personal_loan_active','vehicle_loan_active',
         'personal_loan_closed','vehicle_loan_closed',
         'cc_cons_apr','cc_cons_may','cc_cons_jun',
         'dc_cons_apr','dc_cons_may','dc_cons_jun'
         ]



  

In [130]:
final.drop(labels=extra,axis=1,inplace=True)

In [131]:
final.head(3)

,customer_id,account_type,gender,age,income,emp_tenure_years,tenure_with_bank,region_code,net_banking_flag,avg_days_between_transaction,cc_count_apr,cc_count_may,cc_count_jun,dc_count_apr,dc_count_may,dc_count_jun,card_lim,debit_count_apr,credit_count_apr,credit_count_may,debit_count_may,credit_count_jun,debit_count_jun,loan_enq,emi_active,cc_cons_target,investment,Total_debit_amount,Total_credit_amount,Total_max_credit_acmout,Totat_active_loan,Total_closed_loan,creditcard_exp,debitcard_exp
0,19427,current,M,63,MEDIUM,30.1,10,628.0,1,5.0,8.0,23,13,47,3,30,519000,63.0,1,5,14,40,96,Y,2646.72,NaN,114544.0,231631.92,475518.75,114992.00,2,2.0,24433.29,8292.58
1,16150,current,M,36,MEDIUM,14.4,10,656.0,0,12.0,19.0,32,7,1,37,16,402000,10.0,1,4,63,7,12,Y,5469.79,NaN,1143923.7,210621.79,496597.52,128400.00,2,2.0,35898.73,13916.00
2,11749,current,F,28,MEDIUM,4.8,10,314.0,1,13.0,12.0,30,44,22,15,37,114000,55.0,33,6,82,16,42,Y,7207.85,NaN,281087.6,281651.29,110466.00,1077812.79,2,2.0,33973.99,71489.30


### Profile report
After importing the data, the first tasks is to detect and treat outliers and balnk values. For that reason we will first generate a profile report of our dataframe by using pandas profiling package.\
Some info about the profiling package from their [website](https://pandas-profiling.github.io/pandas-profiling/docs/master/index.html):
pandas_profiling extends the pandas DataFrame with df.profile_report() for quick data analysis.For each column the following statistics - if relevant for the column type - are presented in an interactive HTML report:
- Type inference: detect the types of columns in a dataframe.
- Essentials: type, unique values, missing values
- Quantile statistics like minimum value, Q1, median, Q3, maximum, range, interquartile range
- Descriptive statistics like mean, mode, standard deviation, sum, median absolute deviation, coefficient of variation, kurtosis, skewness
- Most frequent values
- Histogram
- Correlations highlighting of highly correlated variables, Spearman, Pearson and Kendall matrices
- Missing values matrix, count, heatmap and dendrogram of missing values
- Text analysis learn about categories (Uppercase, Space), scripts (Latin, Cyrillic) and blocks (ASCII) of text data.
- File and Image analysis extract file sizes, creation dates and dimensions and scan for truncated images or those containing EXIF information.



#### Generating profile report


In [72]:
# profile_report = ProfileReport(final)
# profile_report.to_widgets()

### EDA

In [74]:
final.select_dtypes(include=['int64','float64'])

,emp_tenure_years,region_code,avg_days_between_transaction,cc_count_apr,debit_count_apr,emi_active,cc_cons_target,investment,Total_debit_amount,Total_credit_amount,Total_max_credit_acmout,Total_closed_loan,creditcard_exp,debitcard_exp
0,30.1,628.0,5.0,8.0,63.0,2646.72,NaN,114544.00,231631.92,475518.75,114992.00,2.0,24433.29,8292.58
1,14.4,656.0,12.0,19.0,10.0,5469.79,NaN,1143923.70,210621.79,496597.52,128400.00,2.0,35898.73,13916.00
2,4.8,314.0,13.0,12.0,55.0,7207.85,NaN,281087.60,281651.29,110466.00,1077812.79,2.0,33973.99,71489.30
3,9.6,614.0,19.0,24.0,32.0,591.34,NaN,205434.25,82508.33,336450.35,215182.00,2.0,22919.31,17534.18
4,12.0,750.0,18.0,100.0,67.0,2621.39,NaN,140601.15,289285.89,168824.00,108610.04,2.0,38417.74,34300.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19983,32.2,354.0,7.0,5.0,29.0,6334.20,NaN,417009.50,372903.21,148587.00,97040.00,2.0,74132.74,13394.04
19984,19.8,809.0,14.0,1.0,5.0,102216.02,NaN,1776525.25,126608.23,157606.00,271850.00,2.0,21791.77,8261.00
19985,18.4,466.0,10.0,6.0,11.0,1930.19,NaN,145830.52,329834.38,121357.00,97637.00,2.0,13841.01,97476.49
19986,6.0,619.0,17.0,5.0,5.0,24499.91,NaN,66244.43,111379.12,145093.76,115984.00,2.0,35217.81,16116.46


### Splitting the dataframe in df_predict and df_train for the data to predict and data to train on respectively

Splitting the rows into df_predict and df_train. df_predict contains cc_cons_target having NA values. These are the customers that we have to predit consumption for.

df_train contains all rows that have a numerical value in cc_cons_target. this df will train out liner regression algo

In [132]:
#filtering all the rows from final that contain NaN values in cc_cons_target and storing those rows in df_predict.
df_predict = final[final.cc_cons_target.isna()]
#dropping the NaN values of cc_cons_target 
df_predict.drop(labels='cc_cons_target',axis=1,inplace=True)

# droppping  all the rows that are in the df_predict dataframe and storing the result in df_train
df_train = final.drop(df_predict.index)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [133]:
df_train.shape

(14995, 34)

### Encoding categorical variables of final_train

In [134]:
#using pd.get_dummies to encode categorical variables in continunous variables and replace the categorical variables.
df_train = pd.get_dummies(df_train)
df_train.loc[:,['account_type_current','account_type_saving','gender_F','gender_M','income_HIGH','income_LOW','income_MEDIUM','loan_enq_Y']]

,account_type_current,account_type_saving,gender_F,gender_M,income_HIGH,income_LOW,income_MEDIUM,loan_enq_Y
104,1,0,0,1,0,0,1,1
105,1,0,0,1,1,0,0,1
106,1,0,0,1,0,0,1,1
107,1,0,1,0,0,0,1,1
108,1,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...
15094,1,0,1,0,0,1,0,1
15095,1,0,0,1,0,0,1,1
15096,1,0,0,1,0,0,1,1
15097,1,0,0,1,0,0,1,1


### Outlier Removal

To remove outliers we use scipy.stats.mstats.winsorize function.

Winsorizing or winsorization is the transformation of statistics by limiting extreme values in the statistical data to reduce the effect of possibly spurious outliers. It is named after the engineer-turned-biostatistician Charles P. Winsor (1895–1951)


In [135]:
#outlier removal with limits:
#     lower limit: 5% of range
#     upper limit: 95% of range
from scipy.stats import mstats

for x in df_train.columns:
    df_train[x] = mstats.winsorize(df_train[x], limits=[0.05, 0.05])

## applying the same aboe steps of missing value removal and outlier imputation to df_predict

In [136]:
df_predict = pd.get_dummies(df_predict)
df_predict.loc[:,['account_type_current','account_type_saving','gender_F','gender_M','income_HIGH','income_LOW','income_MEDIUM','loan_enq_Y']]

,account_type_current,account_type_saving,gender_F,gender_M,income_HIGH,income_LOW,income_MEDIUM,loan_enq_Y
0,1,0,0,1,0,0,1,1
1,1,0,0,1,0,0,1,1
2,1,0,1,0,0,0,1,1
3,1,0,0,1,0,0,1,1
4,1,0,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...
19983,1,0,1,0,0,0,1,1
19984,1,0,0,1,0,0,1,1
19985,1,0,1,0,0,0,1,1
19986,1,0,0,1,0,1,0,1


In [137]:
for x in df_predict.columns:
    df_predict[x] = mstats.winsorize(df_predict[x], limits=[0.05, 0.05])

### Standardization

## Implementing a linear regression model

In [ ]:
##df_train.columns.difference(['cc_cons_target'])

### Splitting x and y teach data into train and test data for model building and model testing

In [141]:
df_train_x = df_train[df_train.columns.difference(['cc_cons_target'])]
df_train_y = df_train[['cc_cons_target']]


x_train, x_test, y_train, y_test = train_test_split(df_train_x, df_train_y, test_size=0.25, random_state=72)

### Scaling Data


In [156]:
# from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler


# scaler = StandardScaler()
# X = scaler.fit_transform(x_train)
# x_train = pd.DataFrame(X, columns=x_train.columns)

# x_train

# sqrt of whole dataset
np.log(x_train)


# for x in x_train.columns:
#   if x_train[x].mean() > 1000:
#     x_train[x]=np.log(x_train[x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [163]:
y_train = np.log(y_train)
y_test = np.log(y_test)

### RFE and selecting features

In [162]:
from sklearn.tree import DecisionTreeRegressor
dt_reg = DecisionTreeRegressor(random_state=0)

lr = LinearRegression()
rfe = RFE(estimator=dt_reg, n_features_to_select=10)
rfe.fit(x_train, y_train)

rfe.ranking_


# selecting features given by RFE
f = rfe.get_support(1) #the most important features
x_train_select = x_train[x_train.columns[f]] # final features`
x_train_select.head(2)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,Total_credit_amount,Total_max_credit_acmout,card_lim,creditcard_exp,customer_id,debitcard_exp,emi_active,emp_tenure_years,investment,region_code
8517,12.078381,11.946835,11.512925,10.231765,9.761463,9.731987,6.947457,20.0,14.227394,799.0
665,12.202504,12.197637,13.186277,10.168080,9.725556,8.923991,8.986941,10.8,13.316346,685.0


In [164]:
#selecting the same features in test data
x_test_select = x_test[x_test.columns[f]]
x_test_select.head(2)

,Total_credit_amount,Total_max_credit_acmout,card_lim,creditcard_exp,customer_id,debitcard_exp,emi_active,emp_tenure_years,investment,region_code
6853,203388.25,223559.11,327000,31761.18,10825,22614.60,4271.29,13.5,205673.25,867.0
1323,315668.77,91587.00,510000,105645.91,1043,42194.54,6141.61,10.5,409703.00,249.0


### Select k best - not used


In [54]:
from sklearn.feature_selection import SelectKBest, f_regression

x_train_select = SelectKBest(f_regression, k=5).fit_transform(x_train, y_train)
x_train_select.shape


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(11246, 5)

### OLS regression

In [165]:
### OLS regression
import statsmodels.api as sm
# y_train = y_train.reindex(x_train.index)
mod = sm.OLS(y_train,x_train_select)

res = mod.fit()

print(res.summary())



#best ols results with np.log(y_train) and rfe with DTreg features = 10
#best ols results with np.log(y_train) and rfe with DTreg features = 15

                                 OLS Regression Results                                
Dep. Variable:         cc_cons_target   R-squared (uncentered):                   0.995
Model:                            OLS   Adj. R-squared (uncentered):              0.995
Method:                 Least Squares   F-statistic:                          2.269e+05
Date:                Fri, 20 Aug 2021   Prob (F-statistic):                        0.00
Time:                        14:25:05   Log-Likelihood:                          4725.6
No. Observations:               11246   AIC:                                     -9431.
Df Residuals:                   11236   BIC:                                     -9358.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------

### Applying liner regresssion on selected features

In [166]:
lr = LinearRegression()
# training the linear regression model on x_train_select and y_train 
lr.fit(x_train_select, y_train.iloc[:,0])

# predicting the values of x_train_select and storing them in y_hat_train
y_hat_train = np.log(lr.predict(x_train_select))

## Checking the accuracy of the train data predictions
#This is a value between [0 = no-fit] and [1 = perfect fit ]
r_sq = met.r2_score(y_pred=y_hat_train,y_true= y_train.iloc[:,0])
print('The accuracy of the model in r^2 :',r_sq)


#mae
mae = met.mean_absolute_error(y_pred=y_hat_train,y_true= y_train)
print('The accuracy of the model in mae :',mae)


# #MAPE
# y_diff = y_hat_train - y_train
# # y_diff  = np.abs(y_diff)/np.max(y_train)#/ np.maximum(np.abs(y_diff))

# maperror = (np.sum(np.abs((y_hat_train - y_train))/y_train))/y_train.count()*100
# print('The accuracy of the model in mape:',maperror,'%')



The accuracy of the model in r^2 : -92.4089811295283
The accuracy of the model in mae : 1.444292945834555


### Predicting the values for test dataset and checking the accuracy of the model

In [168]:
y_hat_test = lr.predict(x_test_select)

# checking the accuracy of test_data predictions
r_sq = met.r2_score(y_pred=y_hat_test, y_true= y_test)
print('The accuracy of the model in r^2 :',r_sq)

met.mean_squared_error(y_pred=y_hat_test, y_true= y_test)

# mae = met.mean_absolute_error(y_pred=y_hat_test,y_true= y_test)

# # #mape calculations
# y_diff = y_test - y_hat_test
# y_diff  = np.abs(y_diff)/np.abs(y_test)
# print('The accuracy of the model in mape:',np.average(y_diff)*100,'%')

# print('The accuracy of the model in mae :',mae)

The accuracy of the model in r^2 : -443674.9410112742


998229.9467211296

In [63]:
(np.sum(np.abs((y_hat_train - y_train))/y_train))/y_train.count()

ValueError: ignored